In [2]:
from VariationalBayes.Parameters import ModelParamsDict
from VariationalBayes.MultinomialParams import SimplexParam
from VariationalBayes.NormalParams import MVNParam

import math

from autograd import grad, hessian, jacobian, hessian_vector_product
import autograd.numpy as np

import copy
import scipy
from scipy import optimize
from scipy import stats

In [34]:
# Number of data points:
n = 100

# Dimension of observations:
d = 2

# Number of clusters:
k = 5

true_pi = np.linspace(0.2, 0.8, k)
true_pi = true_pi / np.sum(true_pi)

true_z = np.random.multinomial(1, true_pi, 100)


mu_prior_mean = np.full(d, 0.)
mu_prior_cov = np.diag(np.full(d, 3 ** 2))

true_mu = np.random.multivariate_normal(mu_prior_mean, mu_prior_cov, k)



[[-4.35176871  1.6629561 ]
 [ 3.2106205  -1.92051036]
 [-1.48497829 -3.38221504]
 [-3.20726262  0.40761483]
 [ 8.04223285 -3.1298826 ]]


In [ ]:


# Build an object to contain a variational approximation to a K-dimensional multivariate normal.
mvn_par = ModelParamsDict()

mvn_par.push_param(VectorParam('e_mu', K))
mvn_par.push_param(VectorParam('var_mu', K, lb=0))

mvn_par['e_mu'].set(np.full(K, 0.1))
mvn_par['var_mu'].set(np.full(K, 2.))

# Generate data
true_mu = np.random.rand(K) - 0.5
x_mat = np.full([N, K], float('nan'))
y_vec = np.full([N], float('nan'))
x_mat = np.random.random(K * N).reshape(N, K) - 0.5
# true_rho = Logistic(np.dot(x_mat[n, :], true_mu))
true_rho = Logistic(np.matmul(x_mat, true_mu))
y_vec = np.random.random(N) < true_rho
